In [ ]:
!pip install spacy
!pip install transformers
!pip install pycountry
!pip install tldextract
!pip install python-whois
!pip install ipinfo
!pip install requests
!pip install matplotlib
!pip install seaborn
!pip install numpy
!pip install pandas
!pip install glob2
!pip install nltk
!pip install pytz
!pip install dateparser
!pip install langdetect
!pip install lingua-language-detector
!pip install requests beautifulsoup4 tldextract pycountry
!pip install python-whois

# Downloading spaCy language models
!python -m spacy download xx_ent_wiki_sm
!python -m spacy download en_core_web_sm

# Downloading NLTK stopwords
import nltk
nltk.download('stopwords')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993221 sha256=9497e530547413aa723677313fb210a911897ff07c262587e5e9de75cf0e6757
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.2/349.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 77.8 MB/s eta 0:00:00
✔ Download and installat

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
!pip install requests beautifulsoup4 tldextract pycountry python-whois geoip2
!pip install stopwords



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 5.7 MB/s eta 0:00:00


In [ ]:
import requests
import re
import json
import os
import datetime
from dateutil import parser
import hashlib
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords as nltk_stopwords
from langdetect import detect
import spacy
import glob
import tldextract
import pycountry
from urllib.parse import urlparse
import socket
import whois
from transformers import MarianMTModel, MarianTokenizer
from lingua import LanguageDetectorBuilder
from bs4 import BeautifulSoup
import nltk

from typing import List, Dict, Any,Tuple

import logging

import nltk
from stopwords import get_stopwords



In [ ]:
# Load the spacy model for multi-language support
nlp = spacy.load("en_core_web_sm")
multi_lang_nlp = spacy.blank("xx")

# ***Remove duplicates function***

In [ ]:
# Function to hash a row
def hash_row(row: Dict[str, Any]) -> str:
    return hashlib.md5(json.dumps(row, sort_keys=True).encode()).hexdigest()

# Function to process a list of JSON dictionaries and remove duplicates
def process_json_files(data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    try:
        unique_data = []
        existing_hashes = set()

        for item in data:
            item_hash = hash_row(item)
            if item_hash not in existing_hashes:
                unique_data.append(item)
                existing_hashes.add(item_hash)

        return unique_data
    except Exception as e:
        print(f"An error occurred while processing data: {e}")
        return data

# ***keywords extraction function***

In [ ]:


# ISO language codes to NLTK stopwords language names
LANGUAGE_CODE_MAP = {
    'en': 'english',
    'de': 'german',
    'fr': 'french',
    'es': 'spanish',
    'it': 'italian',
    'nl': 'dutch',
    'pt': 'portuguese',
    'ru': 'russian',
    'sv': 'swedish',
    'no': 'norwegian',
    'da': 'danish',
    'fi': 'finnish',
    'pl': 'polish',
    'cs': 'czech',
    'sk': 'slovak',
    'hu': 'hungarian',
    'ro': 'romanian',
    'el': 'greek',
}

# Function to detect language
def detect_language(query: str) -> str:
    try:
        return detect(query)
    except Exception as e:
        print(f"Error detecting language: {e}")
        return 'en'  # Default to English if detection fails

# Function to get stop words for a given language
def get_stop_words(language_code: str) -> set:
    try:
        language = LANGUAGE_CODE_MAP.get(language_code, None)
        if language:
            return set(nltk_stopwords.words(language))
        else:
            return set(get_stopwords(language_code))
    except Exception as e:
        print(f"Error getting stop words: {e}")
        return set(nltk_stopwords.words('english'))  # Default to English stop words if language is not supported

# Function to preprocess text by removing stop words
def preprocess_text(text: str, language_code: str) ->  Tuple[str, str]:
    try:
        stop_words = get_stop_words(language_code)
        tokens = [word for word in text.lower().split() if word not in stop_words]
        return ' '.join(tokens), stop_words
    except Exception as e:
        print(f"Error preprocessing text: {e}")
        return text, set()

# Function to extract keywords
def extract_keywords(text: str, num_keywords: int = 10) -> List[str]:
    try:
        tokens = text.split()
        word_freq = Counter(tokens)
        keywords = word_freq.most_common(num_keywords)
        return [keyword[0] for keyword in keywords]
    except Exception as e:
        print(f"Error extracting keywords: {e}")
        return []

# Extract keywords for each product's content and update the JSON data
def keywords_extraction(data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    try:
        for product in data:
            content = product.get("content", "")
            language = detect_language(content)
            preprocessed_content, stop_words = preprocess_text(content, language)
            keywords = extract_keywords(preprocessed_content)
            product['keywords'] = keywords  # Add keywords as a new field
        return data
    except Exception as e:
        print(f"Error in keywords extraction: {e}")
        return data


# ***Inactive products detection fucntion***


In [ ]:
def detect_inactive_products(data: List[Dict[str, Any]], months_threshold: int) -> List[Dict[str, Any]]:
  """
  Detect inactive products based on the last updated date.

  Parameters:
  data (List[Dict[str, Any]]): List of dictionaries where each dictionary represents a product with various attributes.
  months_threshold (int): The threshold in months to determine if a product is inactive.

  Returns:
  List[Dict[str, Any]]: The input list with an additional key 'inactive_product' in each product dictionary indicating
                        whether the product is inactive (True) or not (False).

  Each product dictionary is expected to have the following structure:
  {
      'id': Any,              # Unique identifier for the product
      'updated_at': str,      # ISO 8601 date string indicating the last update time of the product
      ...                     # Other product attributes
  }

  The function adds a new key 'inactive_product' to each dictionary:
  - 'inactive_product': bool - True if the product has not been updated within the specified threshold, False otherwise.
  """

  # Calculate the threshold date by subtracting the specified number of months from the current date
  threshold_date = datetime.datetime.now() - datetime.timedelta(days=months_threshold * 30)

  for product in data:
    # Get the 'updated_at' value from the product dictionary
    updated_at = product.get('updated_at')

    if updated_at:
        try:
          # Parse the 'updated_at' date string to a datetime object
          updated_at = datetime.datetime.strptime(updated_at, '%Y-%m-%dT%H:%M:%S%z')
          updated_at = updated_at.replace(tzinfo=None)  # Remove timezone info

          # Determine if the product is inactive based on the threshold date
          product['inactive_product'] = updated_at < threshold_date
        except ValueError:
          # Handle invalid date format by printing an error message and marking the product as active
          print(f"Invalid date format for product: {product.get('id')}")
          product['inactive_product'] = False
    else:
      # Mark product as inactive if 'updated_at' is missing
      product['inactive_product'] = True

  # Return the updated list of products
  return data

# ***Price standardization***

In [ ]:
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

def get_conversion_rates():
    API_KEY = 'fb40b49fb6b179ac81766c0f'
    url = f'https://v6.exchangerate-api.com/v6/{API_KEY}/latest/USD'
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        if data['result'] == 'success':
            return data['conversion_rates']
        else:
            logging.error("Failed to fetch conversion rates.")
            return None
    except requests.exceptions.RequestException as e:
        logging.error(f"Request error: {e}")
        return None

def convert_to_usd(price_str, conversion_rates):
    try:
        # Dictionary for currency symbols
        currency_symbols = {
            '€': 'EUR',
            '£': 'GBP',
            '¥': 'JPY',
            '$': 'USD',
            'C$': 'CAD',
            'A$': 'AUD',
            'Fr.': 'CHF',
            '¥': 'CNY',
            'د.ت': 'TND',
            'د.إ': 'AED',
            'ر.س': 'SAR',
            'د.ك': 'KWD',
            'ر.ق': 'QAR',
            'ر.ع': 'OMR'
            # Add more symbols as needed
        }

        # Regular expression to match price and currency
        match = re.search(r'(\d+(?:\.\d+)?)\s*([\D]+)?', price_str)
        if not match:
            logging.error(f"Invalid price format: {price_str}")
            return None, None, None

        amount = float(match.group(1))
        currency_identifier = match.group(2).strip() if match.group(2) else 'USD'

        # Convert symbol to currency code if necessary
        currency = currency_symbols.get(currency_identifier, currency_identifier)

        if currency in conversion_rates:
            if currency != 'USD':
                usd_price = amount / conversion_rates[currency]
                return round(usd_price, 2), 'USD', currency
            else:
                return amount, 'USD', currency
        else:
            logging.warning(f"Unknown currency: {currency}. Assuming USD.")
            return amount, 'USD', currency
    except Exception as e:
        logging.error(f"Error converting to USD: {e}")
        return None, None, None

def process_products_for_usd(products: List[Dict[str, Any]], conversion_rates: Dict[str, float]) -> List[Dict[str, Any]]:
    try:
        processed_products = []
        products_changed = 0
        products_in_usd = 0

        for product in products:
            new_product = product.copy()
            product_changed = False
            for variant in new_product.get('variants', []):
                original_price = variant['price']
                new_price, new_currency, original_currency = convert_to_usd(original_price, conversion_rates)
                if new_price is not None:
                    if original_currency != 'USD':
                        if not product_changed:
                            product_name = new_product.get('name')
                            if product_name:
                                print(f"  Product: {product_name}")
                            else:
                                print(f"  Product: (Name not found - ID: {new_product.get('id', 'ID not found')})")
                            product_changed = True
                        print(f"    Changed {original_price} to {new_price:.2f} USD")
                        variant['price_in_USD'] = f"{new_price:.2f}"
                        products_changed += 1
                    else:
                        products_in_usd += 1
                        variant['price_in_USD'] = f"{original_price}"
                else:
                    logging.error(f"Couldn't convert price: {original_price}")

            new_product['currency'] = 'USD'
            processed_products.append(new_product)

        return processed_products
    except Exception as e:
        logging.error(f"Error processing products for USD: {e}")
        return products

def standardize_prices_to_usd(products: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    try:
        conversion_rates = get_conversion_rates()
        if not conversion_rates:
            logging.error("Conversion rates not available. Exiting.")
            return products

        processed_products = process_products_for_usd(products, conversion_rates)
        return processed_products
    except Exception as e:
        logging.error(f"Error in standardizing prices to USD: {e}")
        return products


# ***Country and region from URL extraction function***
get_ip_geolocation ---> Requests per month 50k ?

In [ ]:
import json
import tldextract
import pycountry
import whois
from urllib.parse import urlparse
from typing import List, Dict, Any

def extract_seller_information(data: List[Dict[str, Any]]) -> List[str]:
    sellers = []
    for product in data:
        if 'product_url' in product:
            sellers.append(product['product_url'])
    return sellers

def get_country_from_tld(url: str) -> Tuple[str, str]:
    ext = tldextract.extract(url)
    tld = ext.suffix
    try:
        country = pycountry.countries.lookup(tld).name
    except LookupError:
        country = None
    return country, None

def get_country_and_region_from_whois(url: str) -> Tuple[str, str]:
    try:
        domain = urlparse(url).netloc
        domain_info = whois.whois(domain)
        country = domain_info.get('country')
        region = domain_info.get('state') or domain_info.get('province')
        return country, region
    except Exception as e:
        print(f"WHOIS lookup failed for {url}: {e}")
        return None, None

def get_country_and_region(url: str) -> Tuple[str, str]:
    country, region = get_country_from_tld(url)
    if country:
        return country, region

    country, region = get_country_and_region_from_whois(url)
    if country:
        return country, region

    return None, None

def update_json_with_location(data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    updated_data = []
    try:
        for product in data:
            seller_url = product.get('product_url', '')
            country, region = get_country_and_region(seller_url)
            product['country'] = country if country else 'Not found'
            product['region'] = region if region else 'Not found'
            updated_data.append(product)
            # Print the results for each URL
            print(f"URL: {seller_url}")
            print(f"Country: {product['country']}")
            print(f"Region: {product['region']}\n")
    except Exception as e:
        print(f"An error occurred while updating the JSON data: {e}")
        return data  # Return the original data with missing fields

    return updated_data

# ***Discount extraction function***

In [ ]:
def extract_discounts_and_add_info(data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Extract discount information from product data and add additional information.

    Parameters:
    data (List[Dict[str, Any]]): List of dictionaries where each dictionary represents a product with various attributes.

    Returns:
    List[Dict[str, Any]]: The input list with additional keys 'discount_code' and 'discounts' in each product dictionary.

    Each product dictionary is expected to have the following structure:
    {
        'id': Any,              # Unique identifier for the product
        'title': str,           # Title of the product
        'variants': List[Dict[str, Any]]  # List of variant dictionaries, each containing 'price' and optionally 'compare_at_price'
        ...                     # Other product attributes
    }

    The function adds new keys to each product dictionary:
    - 'discount_code': str - A fixed discount code 'DISCOUNT2024'.
    - 'discounts': List[Dict[str, Any]] - A list of discount details for each variant with a 'compare_at_price' higher than 'price'.
        Each discount dictionary contains:
        {
            'price': float,                   # Current price of the variant
            'compare_at_price': float,        # Original price of the variant before discount
            'discount': float,                # Discount amount
            'discount_percentage': float      # Discount percentage
        }
    """
    try:
        discounts = []
        for product in data:
            # Add discount code to the product
            product['discount_code'] = 'DISCOUNT2024'

            # Initialize the discounts list for the product
            product['discounts'] = []

            # Iterate over each variant in the product
            for variant in product.get('variants', []):
                price = float(variant['price'])
                compare_at_price = variant.get('compare_at_price')

                # Check if compare_at_price is present and not zero
                if compare_at_price and float(compare_at_price) != 0:
                    compare_at_price = float(compare_at_price)
                    discount = compare_at_price - price
                    discount_percentage = (discount / compare_at_price) * 100

                    # Add discount_percentage to the variant dictionary
                    variant['discount_percentage'] = discount_percentage

                    # Append discount details to the product's discounts list
                    product['discounts'].append({
                        'price': price,
                        'compare_at_price': compare_at_price,
                        'discount': discount,
                        'discount_percentage': discount_percentage
                    })

                    # Append discount details to the overall discounts list
                    discounts.append({
                        'product_id': product['id'],
                        'title': product['title'],
                        'price': price,
                        'compare_at_price': compare_at_price,
                        'discount': discount,
                        'discount_percentage': discount_percentage
                    })
        return data

    except Exception as e:
        # Handle any exceptions that occur during processing
        for product in data:
            if 'discount_code' not in product:
                product['discount_code'] = 'DISCOUNT2024'
            if 'discounts' not in product:
                product['discounts'] = []

        print(f"An error occurred: {e}")
        return data

# ***Min max prices***

In [ ]:
def add_min_max_prices(data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    try:
        for product in data:
            prices = [float(variant["price_in_USD"]) for variant in product.get("variants", [])]
            if prices:
                product["min_price"] = min(prices)
                product["max_price"] = max(prices)
            else:
                product["min_price"] = product["max_price"] = None
    except Exception as e:
        print(f"An unexpected error occurred while processing data: {e}")
        raise
    return data

# ***Clean options function***

In [ ]:
def replace_default_title(products: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    try:
        for product in products:
            if "options" in product:
                for option in product["options"]:
                    option["values"] = [None if value and value.lower() == "default title" else value for value in option["values"]]
            if "variants" in product:
                for variant in product["variants"]:
                    if variant["title"].strip() and variant["title"].lower().strip() == "default title":
                        variant["title"] = product["title"]
                    if variant["option1"].strip() and variant["option1"].lower().strip() == "default title":
                        variant["option1"] = None
                    if variant["option2"].strip() and variant["option2"].lower().strip() == "default title":
                        variant["option2"] = None
                    if variant["option3"].strip() and variant["option3"].lower().strip() == "default title":
                        variant["option3"] = None
        return products
    except Exception as e:
        print(f"An error occurred while replacing default titles: {e}")
        return products  # Returning the original products list in case of an error


# ***Freshness score function***


In [ ]:
# Paramètres configurables
MAX_DAYS = 365
WEIGHTS = {'created': 0.2, 'published': 0.3, 'updated': 0.5}

def parse_date(date_str: str) -> datetime.datetime:
    try:
        dt = parser.parse(date_str)
        # Ensure all datetime objects are timezone-naive (convert to UTC)
        if dt.tzinfo is not None:
            dt = dt.astimezone(datetime.timezone.utc).replace(tzinfo=None)
        return dt
    except Exception as e:
        print(f"Erreur lors du parsing de la date {date_str}: {e}")
        return None

def freshness_score(data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    for product in data:
        try:
            created_date = parse_date(product.get('created_at'))
            published_date = parse_date(product.get('published_at'))
            updated_date = parse_date(product.get('updated_at'))

            scores = []
            now = datetime.datetime.now()

            if created_date:
                days_since_created = (now - created_date).days
                scores.append(WEIGHTS['created'] * (MAX_DAYS - min(days_since_created, MAX_DAYS)) / MAX_DAYS)
            if published_date:
                days_since_published = (now - published_date).days
                scores.append(WEIGHTS['published'] * (MAX_DAYS - min(days_since_published, MAX_DAYS)) / MAX_DAYS)
            if updated_date:
                days_since_updated = (now - updated_date).days
                scores.append(WEIGHTS['updated'] * (MAX_DAYS - min(days_since_updated, MAX_DAYS)) / MAX_DAYS)

            product['freshness_score'] = sum(scores) if scores else None

        except Exception as e:
            print(f"Erreur lors du calcul du score de fraîcheur pour le produit {product.get('id', 'ID inconnu')}: {e}")
            product['freshness_score'] = None

    return data


# ***Base id generation***

In [ ]:
def process_products(data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    try:
        # Generate baseid for each product
        for product in data:
            product_title = product['title']
            vendor = product.get('vendor', 'unknown_vendor')
            product_id = product.get('id', 'no_id')

            # Generate initials and hash for baseid
            title_initials = ''.join([word[0].upper() for word in product_title.split() if word])
            vendor_initials = ''.join([word[0].upper() for word in vendor.split() if word])
            id_hash = hashlib.md5(str(product_id).encode()).hexdigest()[:6]

            baseid = f"{title_initials}{vendor_initials}-{id_hash}"
            product['baseid'] = baseid

            for variant in product.get('variants', []):
                variant['baseid'] = baseid

        return data
    except Exception as e:
        print(f"An error occurred: {e}")
        # Add missing fields to original data if any exception occurs
        for product in data:
            if 'baseid' not in product:
                product['baseid'] = None
            for variant in product.get('variants', []):
                if 'baseid' not in variant:
                    variant['baseid'] = None
        return data

# ***Seller information from products extraction function*** (Unscalable)

In [ ]:
# Function to extract seller URLs from the JSON data
def extract_seller_information(file_path):
    # Load the JSON file
    with open(file_path, 'r') as file:
        data = json.load(file)

    # Extracting the seller URLs from each product
    sellers = {product['product_url'] for product in data}

    return data, sellers

# Function to scrape seller information from URLs
def scrape_seller_info(url):
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to retrieve the webpage: {response.status_code}")
        return None

    soup = BeautifulSoup(response.content, 'html.parser')

    # Example scraping logic (adjust based on actual site structure)
    seller_info = {}

    # Assuming there's a contact page link in the footer or header
    contact_link = soup.find('a', href=True, text=re.compile('Contact', re.IGNORECASE))
    if contact_link:
        contact_page_url = contact_link['href']
        if not contact_page_url.startswith('http'):
            contact_page_url = url + contact_page_url
        contact_response = requests.get(contact_page_url)
        contact_soup = BeautifulSoup(contact_response.content, 'html.parser')

        # Extract contact details from the contact page
        email = contact_soup.find('a', href=re.compile('mailto:', re.IGNORECASE))
        phone = contact_soup.find('a', href=re.compile('tel:', re.IGNORECASE))
        address = contact_soup.find('p', text=re.compile('address', re.IGNORECASE))

        seller_info['email'] = email['href'].replace('mailto:', '') if email else 'Not found'
        seller_info['phone'] = phone['href'].replace('tel:', '') if phone else 'Not found'
        seller_info['address'] = address.get_text(strip=True) if address else 'Not found'

    # Extract social media links
    social_media = soup.find_all('a', href=True)
    social_links = {link.text.strip(): link['href'] for link in social_media if 'facebook' in link['href'] or 'instagram' in link['href']}
    seller_info['social_media'] = social_links

    return seller_info

# Main function to add seller info to JSON data and save it
def add_seller_info_to_json(file_path):
    data, sellers = extract_seller_information(file_path)

    for product in data:
        seller_url = product['product_url']
        seller_info = scrape_seller_info(seller_url)
        product['seller_info'] = seller_info

    # Save the updated JSON data back to the file
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)




# ***Final function***

In [ ]:
%mkdir data

#Edit

In [ ]:
from typing import List, Dict, Any

#country and region detection
def update_json_with_location(data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    sellers = extract_seller_information(data)

    for product, seller_url in zip(data, sellers):
        try:
            country, region = get_country_and_region(seller_url)
            product['country'] = country if country else 'Not found'
            product['region'] = region if region else 'Not found'
        except Exception as e:
            print(f"Error processing {seller_url}: {e}")
            product['country'] = 'Not found'
            product['region'] = 'Not found'

    return data


#keywords extraction
def keywords_extraction(data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    try:
        for product in data:
            content = product.get("content", "")
            language = detect_language(content)
            preprocessed_content, stop_words = preprocess_text(content, language)
            keywords = extract_keywords(preprocessed_content)
            product['keywords'] = keywords  # Add keywords as a new field
        return data
    except Exception as e:
        print(f"Error in keywords extraction: {e}")
        return data

#duplicate removal
def process_json_files(data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    try:
        unique_data = []
        existing_hashes = set()

        for item in data:
            item_hash = hash_row(item)
            if item_hash not in existing_hashes:
                unique_data.append(item)
                existing_hashes.add(item_hash)

        return unique_data
    except Exception as e:
        print(f"An error occurred while processing data: {e}")
        return data

#min max price
def add_min_max_prices(data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    try:
        for product in data:
            prices = [float(variant["price_in_USD"]) for variant in product.get("variants", [])]
            if prices:
                product["min_price"] = min(prices)
                product["max_price"] = max(prices)
            else:
                product["min_price"] = product["max_price"] = None
    except Exception as e:
        print(f"An unexpected error occurred while processing data: {e}")
        raise
    return data

#base id generation
def process_products(data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    try:
        # Generate baseid for each product
        for product in data:
            product_title = product['title']
            vendor = product.get('vendor', 'unknown_vendor')
            product_id = product.get('id', 'no_id')

            # Generate initials and hash for baseid
            title_initials = ''.join([word[0].upper() for word in product_title.split() if word])
            vendor_initials = ''.join([word[0].upper() for word in vendor.split() if word])
            id_hash = hashlib.md5(str(product_id).encode()).hexdigest()[:6]

            baseid = f"{title_initials}{vendor_initials}-{id_hash}"
            product['baseid'] = baseid

            for variant in product.get('variants', []):
                variant['baseid'] = baseid

        return data

    except Exception as e:
        print(f"An error occurred: {e}")
        # Add missing fields to original data if any exception occurs
        for product in data:
            if 'baseid' not in product:
                product['baseid'] = None
            for variant in product.get('variants', []):
                if 'baseid' not in variant:
                    variant['baseid'] = None
        return data

#clean options and title
'''def replace_default_title(products: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    try:
        for product in products:
            if "options" in product:
                for option in product["options"]:
                    option["values"] = [None if value and value.lower() == "default title" else value for value in option["values"]]
            if "variants" in product:
                for variant in product["variants"]:
                    if variant["title"] and variant["title"].lower() == "default title":
                        variant["title"] = product["title"]
                    if variant["option1"] and variant["option1"].lower() == "default title":
                        variant["option1"] = None
                    if variant["option2"] and variant["option2"].lower() == "default title":
                        variant["option2"] = None
                    if variant["option3"] and variant["option3"].lower() == "default title":
                        variant["option3"] = None
        return products
    except Exception as e:
        print(f"An error occurred while replacing default titles: {e}")
        return products  # Returning the original products list in case of an error'''

def replace_default_title(products: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    try:
        for product in products:
            if "options" in product:
                for option in product["options"]:
                    option["values"] = [None if value and value.lower() == "default title" else value for value in option["values"]]
            if "variants" in product:
                for variant in product["variants"]:
                    try:
                        if variant["title"].strip() and variant["title"].lower().strip() == "default title":
                            variant["title"] = product["title"]
                        if variant["option1"].strip() and variant["option1"].lower().strip() == "default title":
                            variant["option1"] = None
                        if variant["option2"].strip() and variant["option2"].lower().strip() == "default title":
                            variant["option2"] = None
                        if variant["option3"].strip() and variant["option3"].lower().strip() == "default title":
                            variant["option3"] = None
                    except:
                        #handling cases where optione1 is None in data
                        pass
        return products
    except Exception as e:
        print(f"An error occurred while replacing default titles: {e}")
        return products  # Returning the original products list in case of an error



#inactive products detection
def detect_inactive_products(data: List[Dict[str, Any]], months_threshold: int) -> List[Dict[str, Any]]:
    threshold_date = datetime.datetime.now() - datetime.timedelta(days=months_threshold * 30)

    for product in data:
        updated_at = product.get('updated_at')
        if updated_at:
            try:
                updated_at = datetime.datetime.strptime(updated_at, '%Y-%m-%dT%H:%M:%S%z')
                updated_at = updated_at.replace(tzinfo=None)  # Remove timezone info
                product['inactive_product'] = updated_at < threshold_date
            except ValueError:
                print(f"Invalid date format for product: {product.get('id')}")
                product['inactive_product'] = False
        else:
            product['inactive_product'] = True

    return data

#discount extraction
def extract_discounts_and_add_info(data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    try:
        discounts = []
        for product in data:
            # Add discount code
            product['discount_code'] = 'DISCOUNT2024'

            # Extract discount information
            product['discounts'] = []
            for variant in product.get('variants', []):
                price = float(variant['price'])
                compare_at_price = variant.get('compare_at_price')
                if compare_at_price and float(compare_at_price) != 0:
                    compare_at_price = float(compare_at_price)
                    discount = compare_at_price - price
                    discount_percentage = (discount / compare_at_price) * 100
                    variant['discount_percentage'] = discount_percentage  # Add discount_percentage to the variant
                    product['discounts'].append({
                        'price': price,
                        'compare_at_price': compare_at_price,
                        'discount': discount,
                        'discount_percentage': discount_percentage
                    })
                    discounts.append({
                        'product_id': product['id'],
                        'title': product['title'],
                        'price': price,
                        'compare_at_price': compare_at_price,
                        'discount': discount,
                        'discount_percentage': discount_percentage
                    })
        return data

    except Exception as e:
        # Add missing fields if an exception occurs
        for product in data:
            if 'discount_code' not in product:
                product['discount_code'] = 'DISCOUNT2024'
            if 'discounts' not in product:
                product['discounts'] = []

        print(f"An error occurred: {e}")
        return data

#price standardization
def standardize_prices_to_usd(products: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    try:
        conversion_rates = get_conversion_rates()
        if not conversion_rates:
            logging.error("Conversion rates not available. Exiting.")
            return products

        processed_products = process_products_for_usd(products, conversion_rates)
        return processed_products
    except Exception as e:
        logging.error(f"Error in standardizing prices to USD: {e}")
        return products

#freshness score
def freshness_score(data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    for product in data:
        try:
            created_date = parse_date(product.get('created_at'))
            published_date = parse_date(product.get('published_at'))
            updated_date = parse_date(product.get('updated_at'))

            scores = []
            now = datetime.datetime.now()

            if created_date:
                days_since_created = (now - created_date).days
                scores.append(WEIGHTS['created'] * (MAX_DAYS - min(days_since_created, MAX_DAYS)) / MAX_DAYS)
            if published_date:
                days_since_published = (now - published_date).days
                scores.append(WEIGHTS['published'] * (MAX_DAYS - min(days_since_published, MAX_DAYS)) / MAX_DAYS)
            if updated_date:
                days_since_updated = (now - updated_date).days
                scores.append(WEIGHTS['updated'] * (MAX_DAYS - min(days_since_updated, MAX_DAYS)) / MAX_DAYS)

            product['freshness_score'] = sum(scores) if scores else None

        except Exception as e:
            print(f"Erreur lors du calcul du score de fraîcheur pour le produit {product.get('id', 'ID inconnu')}: {e}")
            product['freshness_score'] = None

    return data


# def add_seller_info_to_json(data: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
#     # Placeholder function for extracting and scraping seller information
#     return data

def etl_ops(file_path: str) -> None:
    try:
        with open(file_path, 'r',encoding='utf-8', errors='ignore') as file:
            data = json.load(file)

        print("Starting duplicate removal...")
        data = process_json_files(data)

        print("Starting the keywords extraction...")
        data = keywords_extraction(data)

        print("Detecting inactive products...")
        data = detect_inactive_products(data, months_threshold=3)

        print("Starting the country and region detection")
        data = update_json_with_location(data)

        print("Starting discount detection...")
        data = extract_discounts_and_add_info(data)

        print("Starting price standardization...")
        data = standardize_prices_to_usd(data)

        print("Adding min_price and max_price...")
        data = add_min_max_prices(data)

        print("Replacing default titles...")
        data = replace_default_title(data)

        print("starting freshness score calculation")
        data= freshness_score(data)

        print("Starting base id generation...")
        data = process_products(data)



        with open(file_path, 'w', encoding='utf-8') as file:
            json.dump(data, file, indent=4, ensure_ascii=False)

    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except json.JSONDecodeError:
        print(f"Error decoding JSON from file: {file_path}")
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")

# List of files to process
file_paths = [
    '/content/sample_data/products-celebratebytheyard-gift-2024-07-22-00-57.json',
    '/content/sample_data/products-fowers-games-2024-07-22-00-59.json',
    '/content/sample_data/products-obsessories-gift-2024-07-22-00-57.json',
    '/content/sample_data/products-sally-gift-2024-07-22-00-55.json',
    '/content/sample_data/products-talon-games-2024-07-22-00-59.json',
    '/content/sample_data/products-theop-games-2024-07-22-01-00.json'

]

# Execute the processing
for file_name in file_paths:
    etl_ops(file_name)

Starting duplicate removal...
Starting the keywords extraction...
Detecting inactive products...
Starting the country and region detection
Starting discount detection...
Starting price standardization...
Adding min_price and max_price...
Replacing default titles...
Starting base id generation...
starting freshness score calculation
Starting duplicate removal...
Starting the keywords extraction...
Detecting inactive products...
Starting the country and region detection
Starting discount detection...
Starting price standardization...
Adding min_price and max_price...
Replacing default titles...
Starting base id generation...
starting freshness score calculation
Starting duplicate removal...
Starting the keywords extraction...
Detecting inactive products...
Starting the country and region detection


2024-08-04 21:26:03,528 - whois.whois - ERROR - Error trying to connect to socket: closing socket - timed out
ERROR:whois.whois:Error trying to connect to socket: closing socket - timed out
2024-08-04 21:26:13,716 - whois.whois - ERROR - Error trying to connect to socket: closing socket - timed out
ERROR:whois.whois:Error trying to connect to socket: closing socket - timed out
2024-08-04 21:26:23,878 - whois.whois - ERROR - Error trying to connect to socket: closing socket - timed out
ERROR:whois.whois:Error trying to connect to socket: closing socket - timed out


Starting discount detection...
Starting price standardization...
Adding min_price and max_price...
Replacing default titles...
Starting base id generation...
starting freshness score calculation
Starting duplicate removal...
Starting the keywords extraction...
Detecting inactive products...
Starting the country and region detection
Starting discount detection...
Starting price standardization...
Adding min_price and max_price...
Replacing default titles...
Starting base id generation...
starting freshness score calculation
Starting duplicate removal...
Starting the keywords extraction...
Detecting inactive products...
Starting the country and region detection


2024-08-04 21:26:40,075 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [Errno 101] Network is unreachable
ERROR:whois.whois:Error trying to connect to socket: closing socket - [Errno 101] Network is unreachable
2024-08-04 21:26:40,097 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [Errno 101] Network is unreachable
ERROR:whois.whois:Error trying to connect to socket: closing socket - [Errno 101] Network is unreachable
2024-08-04 21:26:50,108 - whois.whois - ERROR - Error trying to connect to socket: closing socket - timed out
ERROR:whois.whois:Error trying to connect to socket: closing socket - timed out
2024-08-04 21:26:53,193 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [Errno 101] Network is unreachable
ERROR:whois.whois:Error trying to connect to socket: closing socket - [Errno 101] Network is unreachable
2024-08-04 21:27:03,215 - whois.whois - ERROR - Error trying to connect to socket: closing

Starting discount detection...
Starting price standardization...
Adding min_price and max_price...
Replacing default titles...
Starting base id generation...
starting freshness score calculation
Starting duplicate removal...
Starting the keywords extraction...
Detecting inactive products...
Starting the country and region detection
Starting discount detection...
Starting price standardization...
Adding min_price and max_price...
Replacing default titles...
Starting base id generation...
starting freshness score calculation
